In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

# iTEBD: tranverse-field Ising model

$$
  H 
  = \sum_i -J \sigma^z_i \sigma^z_{i+1} - h \sigma^x_i
  = \sum_i -J \sigma^z_i \sigma^z_{i+1} - \frac{h}{2} \left( \sigma^x_i I_{i+1} + I_i \sigma^x_{i+1}\right)
  = \sum_i h_{i,i+1}.
$$
$$
  h_{i,i+1} = -J \sigma^z_i \sigma^z_{i+1} -\frac{h}{2} \left( \sigma^x_i I_{i+1} + I_i \sigma^x_{i+1}\right).
$$
By using direct product, one finds:
$$ 
  \sigma^z_i \sigma^z_{i+1} 
  = \left(
    \begin{array}{cccc}
      1 & 0 & 0 & 0 \\
      0 & -1 & 0 & 0 \\
      0 & 0 & -1 & 0 \\
      0 & 0 & 0 & 1
    \end{array}
  \right),
  \sigma^x_i I_{i+1} 
  = \left(
    \begin{array}{cccc}
      0 & 0 & 1 & 0 \\
      0 & 0 & 0 & 1 \\
      1 & 0 & 0 & 0 \\
      0 & 1 & 0 & 0
    \end{array}
  \right),
  I_i \sigma^x_{i+1} 
  = \left(
    \begin{array}{cccc}
      0 & 1 & 0 & 0 \\
      1 & 0 & 0 & 0 \\
      0 & 0 & 0 & 1 \\
      0 & 0 & 1 & 0
    \end{array}
  \right)
$$
and

$$
h_{i,i+1}
  = \left(
    \begin{array}{cccc}
      -J & -\frac{h}{2} & -\frac{h}{2} & 0 \\
      -\frac{h}{2} & +J & 0 & -\frac{h}{2} \\
      -\frac{h}{2}& 0 & +J & -\frac{h}{2} \\
      0 & -\frac{h}{2} & -\frac{h}{2} & -J
    \end{array}
  \right)
$$

The exact energy can be obtained by the following segement of code:

In [2]:
# exact energy for J=1, h
h = 2
f = lambda k,h : -2*np.sqrt(1+h**2-2*h*np.cos(k))/np.pi/2.
E0_exact = integrate.quad(f, 0, np.pi, args=(h,))[0]
print(E0_exact)

-2.127088819946744


# Problem-1
Implement iTEBD algorithm to calcualte the ground state energy. Your answer should depend on
* J, h, chi=D=bond dimension, dt, N

For J=-1.0; g=2; chi=10; d=2; delta=0.005; N=10000; You should get something like
* E_iTEBD = -2.126544821260969
* E_exact = -2.127088819946744

In [13]:
def E_0(J,g,chi,d,delta,N):
    l=np.random.rand(2,chi)
    G=np.random.rand(2,d,chi,chi)
    H=np.array([[J,-g/2,-g/2,0],[-g/2,-J,0,-g/2],[-g/2,0,-J,-g/2],[0,-g/2,-g/2,J]])
    w,v=np.linalg.eig(H)
    U=np.reshape(np.dot(np.dot(v,np.diag(np.exp(-delta*(w)))),np.transpose(v)),(2,2,2,2))
    for step in range(N+1):
        A=np.mod(step,2)
        B=np.mod(step+1,2)
        theta=np.tensordot(np.diag(l[B]),G[A],[1,1])
        theta=np.tensordot(theta,np.diag(l[A],0),[2,0])
        theta=np.tensordot(theta,G[B],[2,1])
        theta=np.tensordot(theta,np.diag(l[B],0),[3,0])
        theta=np.tensordot(theta,U,([1,2],[0,1]))
        theta=np.reshape(np.transpose(theta,(2,0,3,1)),(d*chi,d*chi))
        X,Y,Z=np.linalg.svd(theta)
        Z = Z.T
        l[A,0:chi]=Y[0:chi]/np.sqrt(sum(Y[0:chi]**2))
        X=np.reshape(X[0:d*chi,0:chi],(d,chi,chi))
        G[A]=np.transpose(np.tensordot(np.diag(l[B,:]**(-1)),X,[1,1]),(1,0,2))
        Z=np.transpose(np.reshape(Z[0:d*chi,0:chi],(d,chi,chi)),(0,2,1))
        G[B]=np.tensordot(Z,np.diag(l[B]**(-1)),[2,0])
    f = lambda k,g : -2*np.sqrt(1+g**2-2*g*np.cos(k))/np.pi/2.
     
    print("E_iTEBD=",-np.log(np.sum(theta**2))/delta/2,"E_exact=",integrate.quad(f, 0, np.pi, args=(g,))[0])
E_0(-1,2,10,2,0.005,100000)  

E_iTEBD= -2.1270925728925336 E_exact= -2.127088819946744
